# Backtesting Fibonacci Retracement Post Wave 1 Peak

In [21]:
import requests
import sys
import json
from solders.keypair import Keypair
from solders.transaction import VersionedTransaction 
from solana.rpc.api import Client
from solana.rpc.types import TxOpts
from pprint import pprint
from functools import lru_cache
from datetime import datetime, timedelta
import pandas as pd
import json
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML
import os
from datetime import datetime

# Path to Master Functions and Keys
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..', '..'))
sys.path.append(root_dir)

import Birdeye.Basics.Basics_Mastered as birdeye_functions

# Wallet and API Key

In [22]:
API_Key = birdeye_functions.API_Key
wallet = birdeye_functions.wallet
chain = birdeye_functions.chain # Choose between: solana,ethereum,arbitrum,avalanche,bsc,optimism,polygon,base,zksync


# Filter Tokens by Market Cap Range

#### Established Tokens

In [23]:
'''
sort_by = "mc"  # mc, v24hUSD, v24hChangePercent 
sort_type = "desc"  # asc, desc
min_liquidity = 100000  # Minimum liquidity in USD (100k)
min_volume_24h = 1000000  # Minimum 24-hour trading volume in USD
min_market_cap = 1000000  # Minimum market cap in USD (1 million)
max_market_cap = 900000000  # Maximum market cap in USD (900 million)
total_tokens = 300  # Change this to the number of tokens you want to retrieve

# Get the list of established tokens
established_tokens_filtered = birdeye_functions.get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens, chain, API_Key)
#print(established_tokens_filtered)

# Set the index to 'Address'
established_tokens_filtered = established_tokens_filtered.set_index('Address')

# Get the list of addresses
established_tokens_addresses = established_tokens_filtered.index.tolist()

# Get Token Overview Data for established tokens
established_tokens_data = birdeye_functions.get_token_trade_data_multi(established_tokens_addresses, API_Key)

# Get a list of all unique attributes across all DataFrames
all_attributes = set()
for token, df in established_tokens_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        all_attributes.update(df['Attribute'].unique())

# Convert the set to a sorted list
columns_array = sorted(list(all_attributes))

# Create the master DataFrame
established_tokens_master_overview = pd.DataFrame(index=established_tokens_data.keys(), columns=columns_array)

# Fill the DataFrame with values
for token, df in established_tokens_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        for _, row in df.iterrows():
            established_tokens_master_overview.at[token, row['Attribute']] = row['Value']

# Set the index name to 'Address'
established_tokens_master_overview.index.name = 'Address'

# Optional: Save to CSV
established_tokens_master_overview.to_csv('Data/established_tokens_master_overview.csv')
'''

'\nsort_by = "mc"  # mc, v24hUSD, v24hChangePercent \nsort_type = "desc"  # asc, desc\nmin_liquidity = 100000  # Minimum liquidity in USD (100k)\nmin_volume_24h = 1000000  # Minimum 24-hour trading volume in USD\nmin_market_cap = 1000000  # Minimum market cap in USD (1 million)\nmax_market_cap = 900000000  # Maximum market cap in USD (900 million)\ntotal_tokens = 300  # Change this to the number of tokens you want to retrieve\n\n# Get the list of established tokens\nestablished_tokens_filtered = birdeye_functions.get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens, chain, API_Key)\n#print(established_tokens_filtered)\n\n# Set the index to \'Address\'\nestablished_tokens_filtered = established_tokens_filtered.set_index(\'Address\')\n\n# Get the list of addresses\nestablished_tokens_addresses = established_tokens_filtered.index.tolist()\n\n# Get Token Overview Data for established tokens\nestablished_tokens_data = birdeye_functio

#### New Token Listings Summary + OHLCV

In [24]:
import os
from datetime import datetime
import pandas as pd

# Filters for new tokens
days_back = 0  # 0 for last 24 hours, 1 for 24-48 hours ago, 2 for 48-72 hours ago (max 2)
hours_back = 1  # 0-23 hours back within the selected day
minutes_back = 5  # 0-59 minutes back within the selected hour

new_token_liquidity_filter = 10000

# New token specific filters
new_token_min_liquidity = 10000  # Minimum liquidity in USD for new tokens
new_token_max_liquidity = 100000  # Maximum liquidity in USD for new tokens
new_token_min_market_cap = 100000  # Minimum market cap in USD for new tokens
new_token_max_market_cap = 300000  # Maximum market cap in USD for new tokens

# Create the optimized folder structure
data_folder = 'Data'
new_token_data_folder = os.path.join(data_folder, 'New_Token_Data')
current_date = datetime.now().strftime('%Y_%m_%d')  # Format: YYYY_MM_DD
date_folder = os.path.join(new_token_data_folder, current_date)

# Create subfolders for different data types
ohlcv_folder = os.path.join(date_folder, 'OHLCV_Data')
token_summary_folder = os.path.join(date_folder, 'Token_Summary')
live_trades_folder = os.path.join(date_folder, 'Live_Trades')
backtesting_results_folder = os.path.join(date_folder, 'Backtesting_Results')

# Create a single timestamp for both Token_Summary and OHLCV_Data
current_datetime = datetime.now().strftime('%Y_%b_%d_%I%M%p')  # Format: YYYY_Mon_DD_HHMM(AM/PM)
ohlcv_datetime_folder = os.path.join(ohlcv_folder, current_datetime)
token_summary_datetime_folder = os.path.join(token_summary_folder, current_datetime)

# Create the folders if they don't exist
for folder in [data_folder, new_token_data_folder, date_folder, ohlcv_datetime_folder, token_summary_datetime_folder, live_trades_folder, backtesting_results_folder]:
    os.makedirs(folder, exist_ok=True)

# Adjust days_back for the function call
adjusted_days_back = days_back + 1

# Example usage:
new_tokens = birdeye_functions.get_new_listings(adjusted_days_back, hours_back, minutes_back, API_Key, new_token_liquidity_filter)

new_tokens_filtered = new_tokens[1].rename(columns={
    'address': 'Address'
})

# Get List of New Tokens Addresses
new_tokens_address = new_tokens_filtered['Address'].tolist()

# Set Index
new_tokens_filtered = new_tokens_filtered.set_index('Address')

# Get Token Overview Data
token_overview_data = birdeye_functions.get_token_trade_data_multi(new_tokens_address, API_Key)

# Get a list of all unique attributes across all DataFrames
all_attributes = set()
for token, df in token_overview_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        all_attributes.update(df['Attribute'].unique())

# Convert the set to a sorted list
columns_array = sorted(list(all_attributes))

# Create the master DataFrame
new_tokens_master_overview = pd.DataFrame(index=token_overview_data.keys(), columns=columns_array)

# Fill the DataFrame with values
for token, df in token_overview_data.items():
    if isinstance(df, pd.DataFrame) and not df.empty:
        for _, row in df.iterrows():
            new_tokens_master_overview.at[token, row['Attribute']] = row['Value']

# Set the index name to 'Address'
new_tokens_master_overview.index.name = 'Address'

# Filter out rows where 'Error' column is not NaN, if the column exists
if 'Error' in new_tokens_master_overview.columns:
    new_tokens_master_overview = new_tokens_master_overview[new_tokens_master_overview['Error'].isna()]

new_tokens_master_overview["liquidityAddedAt"] = new_tokens_filtered["liquidityAddedAt"]

new_tokens_filtered_overview = new_tokens_master_overview

# Get the list of addresses from new_tokens_master_overview
new_tokens_filtered_overview_address = new_tokens_filtered_overview.index.tolist()

# Get market data for all tokens in new_tokens_master_overview_address
market_data_list = []

for address in new_tokens_filtered_overview_address:
    market_data_df = birdeye_functions.get_token_market_data(address, API_Key)
    if not market_data_df.empty:
        market_data_list.append(market_data_df)

# Combine all market data into a single DataFrame
all_market_data_df = pd.concat(market_data_list, ignore_index=True)

# Set 'Address' as the index
all_market_data_df.set_index('Address', inplace=True)

# Combine all_market_data_df and new_tokens_filtered_overview
new_tokens_mc_added = new_tokens_filtered_overview.join(all_market_data_df, how='left')

# Apply additional filters to new_tokens_mc_added
new_tokens_mc_added = new_tokens_mc_added[
    (new_tokens_mc_added['Liquidity'].astype(float) >= new_token_min_liquidity) &
    (new_tokens_mc_added['Liquidity'].astype(float) <= new_token_max_liquidity) &
    (new_tokens_mc_added['Market Cap'].astype(float) >= new_token_min_market_cap) &
    (new_tokens_mc_added['Market Cap'].astype(float) <= new_token_max_market_cap)
]

# Display the resulting DataFrame
display(new_tokens_mc_added)

# Save the filtered data with market cap range and time range in the filename
mc_range = f"{new_token_min_market_cap/1e6:.1f}M-{new_token_max_market_cap/1e6:.1f}M"
time_range = f"{days_back}d_{hours_back}h_{minutes_back}m"
filename = f"new_tokens_mc_added_filtered_{mc_range}_{time_range}.csv"
file_path = os.path.join(token_summary_datetime_folder, filename)
new_tokens_mc_added.to_csv(file_path)

print(f"Token summary data saved to: {file_path}")


,Error,address,buy_1h,buy_1h_change_percent,buy_24h,buy_24h_change_percent,buy_2h,buy_2h_change_percent,buy_30m,buy_30m_change_percent,...,volume_sell_history_4h_usd,volume_sell_history_8h,volume_sell_history_8h_usd,liquidityAddedAt,Price,Liquidity,Total Supply,Market Cap,Circulating Supply,Circulating Market Cap
Address,,,,,,,,,,,,,,,,,,,,,
eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump,NaN,eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump,3707,None,3719,None,3707,None,2990,317.015342,...,0,0,0,2024-10-20 21:00:42,0.000181,45947.509725,1.000000e+09,181429.851245,1.000000e+09,181429.851245
2Z9QVQoEEURGgdJb1gDADNhrWH4tEGVmc2wxVq2REnHN,NaN,2Z9QVQoEEURGgdJb1gDADNhrWH4tEGVmc2wxVq2REnHN,30,None,30,None,30,None,8,-63.636364,...,0,0,0,2024-10-20 20:55:21,0.000163,35428.489772,9.000000e+08,147036.721381,9.000000e+08,147036.721381
45b6bnQymHKu7z6i9iXpQfn6aJ5pJw2SgeA3KrZZf1FW,NaN,45b6bnQymHKu7z6i9iXpQfn6aJ5pJw2SgeA3KrZZf1FW,21868,None,21968,None,21868,None,13774,70.175439,...,0,0,0,2024-10-20 20:48:04,0.000163,26830.946387,1.000000e+09,163170.765926,1.000000e+09,163170.765926
5qmL9rCSfZ7pBYAsaoeG8SP76ZELeRCK8XtMmYZvpump,NaN,5qmL9rCSfZ7pBYAsaoeG8SP76ZELeRCK8XtMmYZvpump,462,52.980132,764,None,764,None,157,-48.52459,...,0,0,0,2024-10-20 20:39:10,0.000247,60424.338502,1.000000e+09,246730.738074,1.000000e+09,246730.738074


Token summary data saved to: Data/New_Token_Data/2024_10_20/Token_Summary/2024_Oct_20_0946PM/new_tokens_mc_added_filtered_0.1M-0.3M_0d_1h_5m.csv


In [ ]:

# OHLCV Data Retrieval and Storage

# Define the timeframes for OHLCV data (1 hour and under)
timeframes = ['1m', '3m', '5m', '15m', '30m', '1H']

# Get the list of token addresses
token_addresses = new_tokens_mc_added.index.tolist()

# Retrieve and save OHLCV data for each token and timeframe
for address in token_addresses:
    # Create a folder for the token
    token_folder = os.path.join(ohlcv_datetime_folder, address)
    os.makedirs(token_folder, exist_ok=True)
    
    # Retrieve OHLCV data for all timeframes
    ohlcv_data = birdeye_functions.get_ohlcv_data_multi([address], API_Key, timeframes=timeframes)
    
    # Save OHLCV data for each timeframe
    for timeframe, df in ohlcv_data[address].items():
        if not df.empty:
            filename = f"{timeframe}.csv"
            file_path = os.path.join(token_folder, filename)
            df.to_csv(file_path)
            print(f"OHLCV data for {address} ({timeframe}) saved to: {file_path}")
        else:
            print(f"No OHLCV data available for {address} ({timeframe})")

print("OHLCV data retrieval and storage completed.")

# Get Token OHLCV and Save

In [25]:
'''


# Define the timeframes for OHLCV data (1 hour and under)
timeframes = ['1m', '3m', '5m', '15m', '30m', '1H']

# Read the CSV file containing the filtered token data
csv_file_path = file_path  # Using the file_path from the previous code block
filtered_tokens_df = pd.read_csv(csv_file_path)

# Check if 'Address' is a column or an index
if 'Address' in filtered_tokens_df.columns:
    filtered_tokens_df.set_index('Address', inplace=True)
elif filtered_tokens_df.index.name != 'Address':
    raise ValueError("The CSV file does not contain an 'Address' column or index")

# Get the list of token addresses
token_addresses = filtered_tokens_df.index.tolist()

# Create a subfolder for the current date and time
current_datetime = datetime.now().strftime('%Y_%b_%d_%I%M%p')  # Format: YYYY_Mon_DD_HHMM(AM/PM)
ohlcv_datetime_folder = os.path.join(ohlcv_folder, current_datetime)
os.makedirs(ohlcv_datetime_folder, exist_ok=True)

# Retrieve and save OHLCV data for each token and timeframe
for address in token_addresses:
    # Create a folder for the token
    token_folder = os.path.join(ohlcv_datetime_folder, address)
    os.makedirs(token_folder, exist_ok=True)
    
    # Retrieve OHLCV data for all timeframes
    ohlcv_data = birdeye_functions.get_ohlcv_data_multi([address], API_Key, timeframes=timeframes)
    
    # Save OHLCV data for each timeframe
    for timeframe, df in ohlcv_data[address].items():
        if not df.empty:
            filename = f"{timeframe}.csv"
            file_path = os.path.join(token_folder, filename)
            df.to_csv(file_path)
            print(f"OHLCV data for {address} ({timeframe}) saved to: {file_path}")
        else:
            print(f"No OHLCV data available for {address} ({timeframe})")

print("OHLCV data retrieval and storage completed.")

'''


Head of eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump - 1m table:
                                                         address         c  \
datetime                                                                     
2024-10-20 21:00:00  eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump  0.000112   
2024-10-20 21:01:00  eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump  0.000149   
2024-10-20 21:02:00  eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump  0.000161   
2024-10-20 21:03:00  eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump  0.000167   
2024-10-20 21:04:00  eSKEaSVP4fZRSbsZD4yVoRKbSWaTbmpMDq2dUQ9pump  0.000165   

                            h         l         o type             v  
datetime                                                              
2024-10-20 21:00:00  0.000134  0.000065  0.000065   1m  2.380890e+08  
2024-10-20 21:01:00  0.000156  0.000108  0.000112   1m  9.024154e+07  
2024-10-20 21:02:00  0.000181  0.000149  0.000149   1m  2.307119e+07  
2024-10-20 21:03:00  0.000182  0.

# Implement Fibonacci Retracement Optimization